In [1]:
import os

In [2]:
os.getcwd()

'/home/ecbm4040/e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664'

In [3]:
wd_path = r'/home/ecbm4040/e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664'
os.chdir(wd_path)
os.getcwd()

'/home/ecbm4040/e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664'

In [4]:
file_name = 'complete_universe_cleaned_normalized_jonn.csv'

In [5]:
import pandas as pd
import numpy as np

from datetime import datetime

In [6]:
df = pd.read_csv(os.path.join(wd_path,file_name),dtype={'COMNAM':str,'NCUSIP':str,'PERMNO':str,'PERMCO':str,'SICCD':str,'TICKER_PERMNO':str},low_memory=False,index_col=0)

/home/ecbm4040/anaconda3/envs/envTF22/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
df['DATE'] = pd.to_datetime(df['DATE'],format='%Y-%m-%d')

In [8]:
sector_map = {'0':'Agriculture, Forestry and Fishing','10':'Mining','11':'Mining','12':'Mining','13':'Mining','14':'Mining','15':'Construction','16':'Construction','17':'Construction','18':'not used','19':'not used','20':'Manufacturing','21':'Manufacturing','22':'Manufacturing','23':'Manufacturing','24':'Manufacturing','25':'Manufacturing','26':'Manufacturing','27':'Manufacturing','28':'Manufacturing','29':'Manufacturing','30':'Manufacturing','31':'Manufacturing','32':'Manufacturing','33':'Manufacturing','34':'Manufacturing','35':'Manufacturing','36':'Manufacturing','37':'Manufacturing','38':'Manufacturing','39':'Manufacturing','40':'Transportation, Communications, Electric, Gas and Sanitary service','41':'Transportation, Communications, Electric, Gas and Sanitary service','42':'Transportation, Communications, Electric, Gas and Sanitary service','44':'Transportation, Communications, Electric, Gas and Sanitary service','45':'Transportation, Communications, Electric, Gas and Sanitary service','46':'Transportation, Communications, Electric, Gas and Sanitary service','47':'Transportation, Communications, Electric, Gas and Sanitary service','48':'Transportation, Communications, Electric, Gas and Sanitary service','49':'Transportation, Communications, Electric, Gas and Sanitary service','50':'Wholesale Trade','51':'Wholesale Trade','52':'Retail Trade','53':'Retail Trade','54':'Retail Trade','55':'Retail Trade','56':'Retail Trade','57':'Retail Trade','58':'Retail Trade','59':'Retail Trade','60':'Finance, Insurance','61':'Finance, Insurance','62':'Finance, Insurance','63':'Finance, Insurance','64':'Finance, Insurance','65':'Real Estate','66':'Finance, Insurance','67':'Finance, Insurance','70':'Services','71':'Services','72':'Services','73':'Services','74':'Services','75':'Services','76':'Services','78':'Services','79':'Services','80':'Services','81':'Services','82':'Services','83':'Services','86':'Services','87':'Services','89':'Services','91':'Public Administration','92':'Public Administration','94':'Public Administration','95':'Public Administration','96':'Public Administration','97':'Public Administration','99':'Nonclassifiable'}

In [9]:
temp = pd.get_dummies(df['SICCD'].str[:2].replace(sector_map))
df = pd.concat([df, temp], axis=1, sort=False)

In [10]:
import gc

In [11]:
del temp 
gc.collect()
temp = None

In [12]:
df['m+1_returns'] = df['m+1']
df.loc[:,'m+1'] = df.groupby(['DATE'])['m+1'].transform(lambda x: x.rank(pct = True))

In [204]:
identifier_cols = ['DATE','COMNAM','NCUSIP','PERMNO','PERMCO','SICCD','TICKER_PERMNO']
cross_sectional_cols=['accru', 'adv_sale', 'aftret_eq', 'aftret_equity','aftret_invcapx', 'at_tu', 'bm', 'CAPEI', 'capital_rat', 'cash_debt',
       'cash_lt', 'cash_rat', 'cfm', 'curr_debt', 'curr_rat', 'de_rat','debt_assets', 'debt_at', 'debt_capit', 'debt_ebitd', 'debt_invcap',
       'divyield', 'dltt_be', 'equity_invcap', 'evm', 'gpm', 'GProf', 'intcov','intcov_rat', 'invt_act', 'lt_debt', 'lt_ppent', 'npm', 'ocf_lct',
       'opmad', 'opmbd', 'pay_tu', 'pcf', 'pe_ex', 'pe_inc', 'pretret_earnat','pretret', 'profit_lct', 'ps', 'ptb', 'ptpm', 'quick_rat', 'rd_sale',
       'rect_act', 'rect_tu', 'roa', 'roce', 'roe', 'sale_equity','sale_invcap', 'short_debt', 'staff_sale', 'totdebt_invcap']
sector_cols = ['Agriculture, Forestry and Fishing', 'Construction','Finance, Insurance', 'Manufacturing', 'Mining', 'Nonclassifiable',
       'Public Administration', 'Real Estate', 'Retail Trade', 'Services','Transportation, Communications, Electric, Gas and Sanitary service',
       'Wholesale Trade', 'not used']
time_series_cols= ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1']
all_cols = cross_sectional_cols + sector_cols + time_series_cols
Y = ['m+1']

In [205]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

<h1>Regression Model</h1>

In [206]:
'''Hybrid Regression Model'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(83,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(1, activation='sigmoid')(h3)

In [207]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

all_predictions = pd.DataFrame()
dates = df['DATE'].unique()
x = [df[all_cols],df[time_series_cols]]
y = df[Y]

for dt in dates[119:-1]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    test_date = pd.to_datetime(pd.DataFrame({'year':[end_date.year],'month':[end_date.month],'day':[1]})).loc[0] + relativedelta(months=2) - relativedelta(days=1)
    
    print(start_date,end_date)
    print(test_date)
    
    model_file_name = 'models/hybrid_reg_model_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'
    
    hybrid_model=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

    opt=tf.keras.optimizers.Adam(
        learning_rate=0.005,
    )
    hybrid_model.compile(
        optimizer=opt, loss='binary_crossentropy',metrics=['binary_crossentropy','mean_squared_error','mean_absolute_error','mean_absolute_percentage_error']
    )
    hybrid_model.load_weights(model_file_name)
    
    mask_test=(df[['DATE']]==test_date)

    x_test, y_test= [i.loc[mask_test['DATE'],:] for i in x], y.loc[mask_test['DATE']]
    
    predictions = pd.DataFrame(hybrid_model.predict(x_test),index = y_test.index,columns=['Rank Predictions'])
    all_predictions = all_predictions.append(predictions)
    
    del predictions
    del x_test
    del y_test
    del mask_test
    del hybrid_model
    del model_file_name
    gc.collect()
    predictions = None
    x_test = None
    y_test = None
    mask_test = None
    hybrid_model = None
    model_file_name = None

1970-12-31 00:00:00 1980-12-31 00:00:00
1981-01-31 00:00:00
1971-01-31 00:00:00 1981-01-31 00:00:00
1981-02-28 00:00:00
1971-02-28 00:00:00 1981-02-28 00:00:00
1981-03-31 00:00:00
1971-03-31 00:00:00 1981-03-31 00:00:00
1981-04-30 00:00:00
1971-04-30 00:00:00 1981-04-30 00:00:00
1981-05-31 00:00:00
1971-05-31 00:00:00 1981-05-31 00:00:00
1981-06-30 00:00:00
1971-06-30 00:00:00 1981-06-30 00:00:00
1981-07-31 00:00:00
1971-07-31 00:00:00 1981-07-31 00:00:00
1981-08-31 00:00:00
1971-08-31 00:00:00 1981-08-31 00:00:00
1981-09-30 00:00:00
1971-09-30 00:00:00 1981-09-30 00:00:00
1981-10-31 00:00:00
1971-10-31 00:00:00 1981-10-31 00:00:00
1981-11-30 00:00:00
1971-11-30 00:00:00 1981-11-30 00:00:00
1981-12-31 00:00:00
1971-12-31 00:00:00 1981-12-31 00:00:00
1982-01-31 00:00:00
1972-01-31 00:00:00 1982-01-31 00:00:00
1982-02-28 00:00:00
1972-02-28 00:00:00 1982-02-28 00:00:00
1982-03-31 00:00:00
1972-03-31 00:00:00 1982-03-31 00:00:00
1982-04-30 00:00:00
1972-04-30 00:00:00 1982-04-30 00:00:00


In [208]:
all_predictions

,Rank Predictions
564319,0.461458
564320,0.476894
564321,0.316621
564322,0.716326
564323,0.551334
...,...
3059175,0.521581
3059176,0.517337
3059177,0.482406
3059178,0.551045


In [209]:
returns_df = df[['DATE','m+1_returns']].copy()
returns_df = returns_df.merge(all_predictions,left_index=True, right_index=True)

In [210]:
from scipy.stats import zscore
returns_df['ZScore Predictions'] = returns_df.groupby(['DATE'])['Rank Predictions'].transform(lambda x : zscore(x,ddof=1))

In [211]:
returns_dates =  returns_df['DATE'].unique()

In [212]:
returns_df.head(10)

,DATE,m+1_returns,Rank Predictions,ZScore Predictions
564319,1981-01-31,-0.5941,0.461458,-0.660095
564320,1981-01-31,0.0000,0.476894,-0.420227
564321,1981-01-31,-25.0000,0.316621,-2.910840
564322,1981-01-31,1.8018,0.716326,3.300506
564323,1981-01-31,7.6000,0.551334,0.736560
564324,1981-01-31,3.7037,0.530981,0.420271
564326,1981-01-31,4.7170,0.470783,-0.515190
564327,1981-01-31,-6.5487,0.526427,0.349502
564328,1981-01-31,8.8235,0.174881,-5.113473
564329,1981-01-31,41.0435,0.525990,0.342717


<h1>Model Factor Returns</h1>

In [213]:
import statsmodels.api as sm

model_factor_returns = pd.DataFrame()

for dt in returns_dates:
    idx = (returns_df['DATE']==dt)
    
    X = returns_df.loc[idx,'ZScore Predictions']
    y = returns_df.loc[idx,'m+1_returns']
    
    model = sm.OLS(y, X)
    results = model.fit()
    
    model_factor_returns = model_factor_returns.append(pd.DataFrame(results.params[0]/100+1,index=[dt],columns=['Return']))

In [214]:
annual_returns = (model_factor_returns.resample('A').prod()-1)

In [215]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Annual Returns', x=annual_returns.index, y=annual_returns['Return'])
])

fig.show()

<h1>Long Top 4% Short Bottom 4%</h1>

In [216]:
percent = 0.04

model_ls_returns = pd.DataFrame()

for dt in returns_dates:
    idx = (returns_df['DATE']==dt)
    num = int(np.ceil(idx.sum() * percent))
    print(num)
    temp = returns_df.loc[idx].copy().sort_values(['Rank Predictions'])
    short = temp.head(num)
    long = temp.tail(num)
    
    ret = long['m+1_returns'].mean() - short['m+1_returns'].mean()
    model_ls_returns = model_ls_returns.append(pd.DataFrame(ret/100+1,index=[dt],columns=['Return']))

134
135
135
135
137
137
138
139
139
139
140
142
143
146
147
147
148
149
148
149
149
148
149
148
148
149
149
149
151
151
151
153
154
156
157
158
159
161
162
163
166
166
166
167
166
166
166
165
165
166
165
165
166
164
164
164
164
163
163
162
162
163
163
163
165
165
165
166
166
166
166
165
166
168
169
170
172
173
173
174
174
174
176
175
175
175
175
174
175
174
174
173
172
172
171
170
169
170
169
169
170
169
168
167
167
166
166
165
164
166
166
165
167
167
167
166
166
166
166
165
165
165
165
164
165
165
165
166
167
167
167
167
168
168
169
169
171
171
171
172
173
173
173
173
174
176
176
176
179
180
180
182
183
184
186
187
189
195
197
197
205
205
207
209
209
209
223
223
223
226
226
226
228
228
229
230
231
231
232
233
233
234
234
235
238
239
242
242
243
243
244
246
246
249
249
250
252
251
250
250
250
250
250
249
248
249
248
248
248
248
247
246
245
242
241
238
236
236
234
233
233
232
231
231
231
230
230
229
229
230
228
228
230
229
227
227
225
225
226
224
222
222
220
218
217
214
213
210
208
207


In [217]:
annual_returns_2 = (model_ls_returns.resample('A').prod()-1)

In [218]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Annual Returns', x=annual_returns_2.index, y=annual_returns_2['Return'])
])

fig.show()

<h1>Classification Model</h1>

In [219]:
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(83,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(5, activation='softmax')(h3)

In [237]:
all_predictions = pd.DataFrame()
dates = df['DATE'].unique()

X = [df[all_cols],df[time_series_cols]]
y = pd.DataFrame(pd.get_dummies(pd.qcut(df[Y].values[:,0], 5, range(0,5))),index=df.index)

for dt in dates[119:-1]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    test_date = pd.to_datetime(pd.DataFrame({'year':[end_date.year],'month':[end_date.month],'day':[1]})).loc[0] + relativedelta(months=2) - relativedelta(days=1)
    
    print(start_date,end_date)
    print(test_date)
    
    try:
        model_file_name = 'models/hybrid_class_model_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'
    
        hybrid_model=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

        opt=tf.keras.optimizers.Adam(
            learning_rate=0.005,
        )
        hybrid_model.compile(
            optimizer=opt, loss='CategoricalCrossentropy',metrics=['accuracy']
        )
        hybrid_model.load_weights(model_file_name)

        mask_test=(df[['DATE']]==test_date)

        x_test, y_test=[i.loc[mask_test['DATE'],:] for i in x], y.loc[mask_test['DATE']]

        predictions = pd.DataFrame(hybrid_model.predict(x_test),index = y_test.index,columns=[0,1,2,3,4])
        all_predictions = all_predictions.append(predictions)
    except:
        print('error')

1970-12-31 00:00:00 1980-12-31 00:00:00
1981-01-31 00:00:00
1971-01-31 00:00:00 1981-01-31 00:00:00
1981-02-28 00:00:00
1971-02-28 00:00:00 1981-02-28 00:00:00
1981-03-31 00:00:00
1971-03-31 00:00:00 1981-03-31 00:00:00
1981-04-30 00:00:00
1971-04-30 00:00:00 1981-04-30 00:00:00
1981-05-31 00:00:00
1971-05-31 00:00:00 1981-05-31 00:00:00
1981-06-30 00:00:00
1971-06-30 00:00:00 1981-06-30 00:00:00
1981-07-31 00:00:00
1971-07-31 00:00:00 1981-07-31 00:00:00
1981-08-31 00:00:00
1971-08-31 00:00:00 1981-08-31 00:00:00
1981-09-30 00:00:00
1971-09-30 00:00:00 1981-09-30 00:00:00
1981-10-31 00:00:00
1971-10-31 00:00:00 1981-10-31 00:00:00
1981-11-30 00:00:00
1971-11-30 00:00:00 1981-11-30 00:00:00
1981-12-31 00:00:00
1971-12-31 00:00:00 1981-12-31 00:00:00
1982-01-31 00:00:00
1972-01-31 00:00:00 1982-01-31 00:00:00
1982-02-28 00:00:00
1972-02-28 00:00:00 1982-02-28 00:00:00
1982-03-31 00:00:00
1972-03-31 00:00:00 1982-03-31 00:00:00
1982-04-30 00:00:00
1972-04-30 00:00:00 1982-04-30 00:00:00


In [238]:
all_predictions

,0,1,2,3,4
564319,0.188970,0.232948,0.219962,0.213444,0.144677
564320,0.226709,0.205820,0.186717,0.188066,0.192688
564321,0.382587,0.175499,0.142221,0.120617,0.179077
564322,0.060795,0.178873,0.291393,0.340027,0.128911
564323,0.094515,0.189375,0.256226,0.294435,0.165449
...,...,...,...,...,...
1709692,0.308356,0.180870,0.145631,0.146794,0.218349
1709694,0.148400,0.244471,0.246593,0.227236,0.133301
1709695,0.083616,0.246327,0.306078,0.265921,0.098059
1709696,0.337037,0.123561,0.107014,0.113782,0.318605


In [239]:
returns_df_2 = df[['DATE','m+1_returns']].copy()
returns_df_2 = returns_df_2.merge(all_predictions,left_index=True, right_index=True)

In [241]:
returns_dates =  returns_df_2['DATE'].unique()

In [252]:
percent = 0.04

model_ls_returns_2 = pd.DataFrame()

for dt in returns_dates:
    idx = (returns_df_2['DATE']==dt)
    num = int(np.ceil(idx.sum() * percent))
    print(num)
    long = returns_df_2.loc[idx].copy().sort_values([4]).tail(num)
    short = returns_df_2.loc[idx].copy().sort_values([0]).tail(num)
    
    ret = long['m+1_returns'].mean() - short['m+1_returns'].mean()
    model_ls_returns_2 = model_ls_returns_2.append(pd.DataFrame(ret/100+1,index=[dt],columns=['Return']))

134
135
135
135
137
137
138
139
139
139
140
142
143
146
147
147
148
149
148
149
149
148
149
148
148
149
149
149
151
151
151
153
154
156
157
158
159
161
162
163
166
166
166
167
166
166
166
165
165
166
165
165
166
164
164
164
164
163
163
162
162
163
163
163
165
165
165
166
166
166
166
165
166
168
169
170
172
173
173
174
174
174
176
175
175
175
175
174
175
174
174
173
172
172
171
170
169
170
169
169
170
169
168
167
167
166
166
165
164
166
166
165
167
167
167
166
166
166
166
165
165
165
165
164
165
165
165
166
167
167
167
167
168
168
169
169
171
171
171
172
173
173
173
173
174
176
176
176
179
180
180
182
183
184
186
187
189
195
197
197
205
205
207
209
209
209
223
223
223
226
226
226
228
228
229
230
231
231
232
233
233
234
234
235
238
239
242
242
243
243


In [254]:
annual_returns_3 = (model_ls_returns_2.resample('A').prod()-1)

In [255]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Annual Returns', x=annual_returns_3.index, y=annual_returns_3['Return'])
])

fig.show()